In [ ]:
from ase.io import read as ase_read
from ase.io import write as ase_write
import numpy as np
import copy
import json
from collections import Counter

In [ ]:
input_xyz = "jp_dio-orig_min4.xyz"
min_orig_atoms = ase_read(input_xyz)

In [ ]:
with open("noOidx2orig.json", "r") as f:
    index_map = json.load(f)

# I want to reverse this, i.e. go from orig to noO
index_map = {int(v): int(k) for k,v in index_map.items()}

In [ ]:
nnlist_data = np.load("orig_dio_polycrystal_neighborlist10dot4.npz")
nn_i, nn_j, nn_S = nnlist_data["i"] , nnlist_data["j"], nnlist_data["S"]
nn_dict = {}
for k in range(len(nn_i)):
    iidx = int(nn_i[k])
    if iidx not in nn_dict:
        nn_dict[iidx] = []
    nn_dict[iidx].append(int(nn_j[k]))

In [ ]:
grain_ptm_data = np.load("grains_ptm_111025_min4_fixed.npz") # notice! using the fixed version now
noO_grains = grain_ptm_data["grains"]
noO_ptm_types = grain_ptm_data["ptm_types"]

In [ ]:
xyz_grain_idxs = []
xyz_ptm_types = []

for i,atm in enumerate(min_orig_atoms):
    if atm.symbol == "O":
        xyz_grain_idxs.append(-1)
        xyz_ptm_types.append(-1)
        continue

    xyz_grain_idxs.append(noO_grains[index_map[i]])
    xyz_ptm_types.append(noO_ptm_types[index_map[i]])

In [ ]:
def generate_temp_xyz(index):
    min_orig_out = copy.deepcopy(min_orig_atoms)

    isneighbor = np.zeros(len(min_orig_atoms))
    isneighbor[nn_dict[index]] = 1
    isneighbor[index] = 1
    min_orig_out.set_array("is_neighbor", isneighbor)

    new_symbols = min_orig_out.get_chemical_symbols().copy()
    new_symbols[index] = "Np"
    min_orig_out.set_chemical_symbols(new_symbols)

    ref_symbols = min_orig_out.get_chemical_symbols().copy()
    new_symbols = ref_symbols.copy()
    new_symbols[90839] = "Np"
    min_orig_out.set_chemical_symbols(new_symbols)

    min_orig_out.set_array("grain_index", np.array(xyz_grain_idxs))
    min_orig_out.set_array("ptm_type", np.array(xyz_ptm_types))

    ase_write("temp.xyz", min_orig_out, format="extxyz")



In [ ]:
o_idxs = [idx for idx in range(len(min_orig_atoms)) if min_orig_atoms[idx].symbol == "O"]

In [ ]:
oatom_envs = []
for idx in o_idxs[:10]:
    neighbor_idxs = nn_dict[idx]

    local_grains = []
    local_ptm_types = []
    o_count = 0
    for nidx in neighbor_idxs:
        atm = min_orig_atoms[nidx]
        if atm.symbol == "O":
            o_count += 1
            continue
        local_grains.append(int(noO_grains[index_map[nidx]]))
        local_ptm_types.append(int(noO_ptm_types[index_map[nidx]]))
    num_Hf_neighs = len(local_grains)

    num_hcp = local_ptm_types.count(2)
    num_other = local_ptm_types.count(0)
    fract_hcp = num_hcp/num_Hf_neighs
    fract_hcp_c = 1 -fract_hcp
    fract_other = num_other/num_Hf_neighs

    grain_counter = dict(Counter(local_grains))
    grain_fract = {k: v/num_Hf_neighs for k,v in grain_counter.items()}

    oatom_envs.append({"index" : idx,
                       "o_count" : o_count,
                       "fract_hcp" : fract_hcp,
                       "fract_hcp_c" : fract_hcp_c,
                       "fract_other": fract_other,
                       "grain_fract": grain_fract})



In [ ]:
for neigh_summary in oatom_envs:
    print(neigh_summary["grain_fract"])

In [ ]:
oatom_envs[-2]

In [ ]:
generate_temp_xyz(90839)